In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import sqrt
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
%matplotlib inline

In [11]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [12]:
train_df = train_df[['id','precio']]
train_df['precio'].value_counts()

1500000.0     3041
2500000.0     3036
1200000.0     2909
3500000.0     2726
1300000.0     2480
2200000.0     2420
1100000.0     2394
1800000.0     2290
1600000.0     2217
850000.0      2192
750000.0      2163
1250000.0     2159
2300000.0     2108
1350000.0     2103
650000.0      2061
3200000.0     2054
4500000.0     2040
2000000.0     1904
950000.0      1867
1650000.0     1865
2800000.0     1863
1400000.0     1850
550000.0      1820
2100000.0     1802
3000000.0     1791
1700000.0     1763
1850000.0     1760
2600000.0     1735
1150000.0     1722
1750000.0     1702
              ... 
10807000.0       1
468778.0         1
675411.0         1
468768.0         1
7500250.0        1
10815000.0       1
3752490.0        1
469105.0         1
338144.0         1
1352990.0        1
938630.0         1
10822700.0       1
2705670.0        1
1877000.0        1
407500.0         1
1352610.0        1
676300.0         1
2705184.0        1
938426.0         1
1876500.0        1
1352560.0        1
469208.0    

In [13]:
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0)
features['precio'].value_counts()

1500000.0     3041
2500000.0     3036
1200000.0     2909
3500000.0     2726
1300000.0     2480
2200000.0     2420
1100000.0     2394
1800000.0     2290
1600000.0     2217
850000.0      2192
750000.0      2163
1250000.0     2159
2300000.0     2108
1350000.0     2103
650000.0      2061
3200000.0     2054
4500000.0     2040
2000000.0     1904
950000.0      1867
1650000.0     1865
2800000.0     1863
1400000.0     1850
550000.0      1820
2100000.0     1802
3000000.0     1791
1700000.0     1763
1850000.0     1760
2600000.0     1735
1150000.0     1722
1750000.0     1702
              ... 
10807000.0       1
468778.0         1
675411.0         1
468768.0         1
7500250.0        1
10815000.0       1
3752490.0        1
469105.0         1
338144.0         1
1352990.0        1
938630.0         1
10822700.0       1
2705670.0        1
1877000.0        1
407500.0         1
1352610.0        1
676300.0         1
2705184.0        1
938426.0         1
1876500.0        1
1352560.0        1
469208.0    

In [14]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  180000 Test:  60000


In [50]:
parameters = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

In [51]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_test = lgb.Dataset(x_test, y_test, reference=lgb_train)

In [52]:
evals_result = {}  # to record eval results for plotting
lgb = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=[lgb_train, lgb_test],
                feature_name=['f' + str(i + 1) for i in range(x_train.shape[-1])],
                categorical_feature=[21],
                evals_result=evals_result,
                verbose_eval=10)

In [53]:
lgb_grid = GridSearchCV(gbm,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)


In [54]:
lgb_grid.fit(features, labels)

TypeError: estimator should be an estimator implementing 'fit' method, <lightgbm.basic.Booster object at 0x7fd1d817e5c0> was passed